### Filament parameters for Phantom

In [1]:
import numpy as np
import math
from astropy import constants as const
from astropy import units as u
#from astropy.constants import si

In [2]:
densUnit = u.solMass/(u.pc)**3.

In [3]:
densUnit

Unit("solMass / pc3")

In [4]:
filamentM = 36*u.solMass #filament mass
filamentL = 1.5*u.pc #filament length
filamentR = 0.1*u.pc #filament radius
filamentV = math.pi*filamentR**2.*filamentL
filamentDens = filamentM/filamentV
filamentDens.decompose(u.cgs.bases)

<Quantity 5.17029882e-20 g / cm3>

In [5]:
filamentDens.decompose(u.cgs.bases)/30.

<Quantity 1.72343294e-21 g / cm3>

In [6]:
filamentM = 50*u.solMass #filament mass

filamentDens = filamentM/filamentV
filamentDens.decompose(u.cgs.bases)

<Quantity 7.18097058e-20 g / cm3>

In [7]:
filamentM = 20*u.solMass #filament mass

filamentDens = filamentM/filamentV
filamentDens.decompose(u.cgs.bases)

<Quantity 2.87238823e-20 g / cm3>

In [8]:
filamentDens.decompose(u.cgs.bases)/30.

<Quantity 9.57462744e-22 g / cm3>

**Calcualtion of sound speed:**

$\mu$ and temperature are input parameters.

In [9]:
def soundSpeed(mu, T):
    T = T*u.Kelvin
    cs = (const.k_B*T/mu/const.m_p)**(0.5)
    cs = cs.decompose(u.cgs.bases)
    return cs

In [10]:
soundSpeed(2.381, 15)

<Quantity 22803.87651997 cm / s>

But we need to know the sound speed in the code unit when we input initial parameters for the Phantom code.

In [11]:
const.G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>

In [12]:
const.G.unit

Unit("m3 / (kg s2)")

In [13]:
timeUnitSi = (u.m**3/const.G.unit/u.kg)**(0.5); timeUnitSi

Unit("s")

In [14]:
timeUnit = (const.pc**3/(const.G)/const.M_sun)**(0.5)

In [15]:
timeUnit

<Quantity 4.70511238e+14 s>

In [16]:
timeUnit.to(u.year)/10**6

<Quantity 14.90960143 yr>

In [17]:
velocityUnit = u.pc/timeUnit; velocityUnit

<Quantity 2.12534775e-15 pc / s>

In [18]:
soundSpeedDimless = soundSpeed(2.381, 15)/velocityUnit; soundSpeedDimless

<Quantity 1.07294802e+19 cm / pc>

In [19]:
soundSpeedDimless.decompose(u.cgs.bases)

<Quantity 3.47718771>

---
### Calculation of maximum density that can be resolved:

In [20]:
Rg = const.R*u.mol/u.gram; Rg

<Quantity 8.31446262 J / (g K)>

#### Calculation of Jeans length:

In [21]:
filamentM = 72*u.solMass #filament mass

filamentDens = filamentM/filamentV
filamentDens.decompose(u.cgs.bases)

<Quantity 1.03405976e-19 g / cm3>

In [22]:
cs = soundSpeed(2.381, 15)
JeansL = (math.pi*cs**2/const.G/filamentDens)**(1/2.); JeansL

<Quantity 1.26571251e+08 cm kg(1/2) pc(3/2) / (m(3/2) solMass(1/2))>

In [23]:
JeansL.decompose(u.cgs.bases)

<Quantity 4.86528108e+17 cm>

In [24]:
JeansL.decompose(u.cgs.bases).to(u.parsec)

<Quantity 0.15767302 pc>

In [25]:
# function to calculate rho_crit (Bate & Burkert 1997)
def calcRhoCrit (T, mu, Ntot, Mtot):
    T = T * u.Kelvin
    Mtot = Mtot * const.M_sun
    Nnei = 57.9 # M_4 cubic kernel
    rhoCrit = (3./4/math.pi) * (5*Rg*T/2/const.G/mu)**3 * (Ntot/(2*Nnei)/Mtot)**2
    return rhoCrit.decompose(u.cgs.bases)

# function to calculate rho_crit (Hubber et al. 2006)
def calcRhoCritHubber (T, mu, Ntot, Mtot):
    T = T * u.Kelvin
    Mtot = Mtot * const.M_sun
    ss2 = (const.k_B*T/mu/const.m_p) # sound speed ^ 2
    Nnei = 57.9
    rhoCritHubber = (math.pi*ss2/const.G)**3 * (math.pi/(6*Nnei*(Mtot/Ntot)))**2
    return rhoCritHubber.decompose(u.cgs.bases)

---
For non-turbulent filament:

In [26]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 300000, Mtot = 50)

<Quantity 1.22429465e-15 g / cm3>

In [27]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 30000, Mtot = 100)

<Quantity 3.06073664e-18 g / cm3>

In [28]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 1000000, Mtot = 38)

<Quantity 2.35513745e-14 g / cm3>

In [29]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 300000, Mtot = 38)

<Quantity 2.11962371e-15 g / cm3>

In [30]:
calcRhoCritHubber (T = 15, mu = 2.381, Ntot = 300000, Mtot = 50)

<Quantity 1.09198162e-14 g / cm3>

In [31]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 30000, Mtot = 50)

<Quantity 1.22429465e-17 g / cm3>

In [32]:
calcRhoCrit (T = 15, mu = 2.46, Ntot = 3500000, Mtot = 50)

<Quantity 1.51095802e-13 g / cm3>

---

In [33]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 36)

<Quantity 3.21450821e-13 g / cm3>

In [34]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 72)

<Quantity 8.03627053e-14 g / cm3>

In [35]:
calcRhoCrit (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 109)

<Quantity 3.50644108e-14 g / cm3>

In [36]:
calcRhoCritHubber (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 36) # <--- Hubber

<Quantity 2.86710709e-12 g / cm3>

In [37]:
calcRhoCritHubber (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 72) # <--- Hubber

<Quantity 7.16776773e-13 g / cm3>

In [38]:
calcRhoCritHubber (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 109) # <--- Hubber

<Quantity 3.12748993e-13 g / cm3>

---

### Jeans length at critical density:

In [39]:
rhoCrit = calcRhoCrit (T = 15, mu = 2.381, Ntot = 3.5e6, Mtot = 72)
print(rhoCrit)
cs = soundSpeed(2.381, 15)
RJ = (math.pi*cs**2/const.G/rhoCrit)**(1/2.)
print(RJ.decompose(u.cgs.bases).to(u.AU))
JeansMassCrit = 4./3*math.pi*rhoCrit*(RJ/2)**3
print(JeansMassCrit.to(u.solMass))

8.036270529401841e-14 g / cm3
36.89164909326973 AU
0.003557195247915793 solMass


### Jeans length at critical density for sink creation:

In [40]:
rhoSink = rhoCrit
cs = soundSpeed(2.381, 15)
JeansLengthSink = (math.pi*cs**2/const.G/rhoSink)**(1/2.)
print(JeansLengthSink.decompose(u.cgs.bases).to(u.AU))
print(JeansLengthSink.decompose(u.cgs.bases).to(u.pc))

36.89164909326973 AU
0.0001788557619910937 pc


### Critical density for Bate (2009, MNRAS, 397, 232–248) initial condition:

In [41]:
calcRhoCrit (T = 10, mu = 2.46, Ntot = 3500000, Mtot = 50)

<Quantity 4.47691264e-14 g / cm3>

In [42]:
rhoCrit = calcRhoCrit (T = 10, mu = 2.46, Ntot = 3500000, Mtot = 50); print(rhoCrit)
cs = soundSpeed(2.46, 10)
JeansLengthCrit = (math.pi*cs**2/const.G/rhoCrit)**(1/2.)
print(JeansLengthCrit.decompose(u.cgs.bases).to(u.AU))

4.476912642691632e-14 g / cm3
39.70384325976633 AU


### Jeans mass at $\rho=10^{-13} g/cm^3$

In [43]:
rhoCrit = 1e-13 * u.g / (u.cm)**3
cs = soundSpeed(2.46, 10)
RJ = (math.pi*cs**2/const.G/rhoCrit)**(1/2.)
print(JeansLengthCrit.decompose(u.cgs.bases).to(u.AU))
JeansMassCrit = 4./3*math.pi*rhoCrit*(RJ/2.)**3
print(JeansMassCrit.to(u.solMass))

39.70384325976633 AU
0.0016528541077230976 solMass


### Mass per unit length for an isothermal Ostriker filament:

In [44]:
ml = 2*soundSpeed(2.381, 15)**2/const.G; ml

<Quantity 1.55826614e+19 cm2 kg / m3>

In [45]:
ml.decompose(u.cgs.bases)

<Quantity 1.55826614e+16 g / cm>

In [46]:
solarmassperparsec = u.solMass/u.parsec; solarmassperparsec

Unit("solMass / pc")

In [47]:
ml.decompose(u.cgs.bases).to(solarmassperparsec)

<Quantity 24.18166878 solMass / pc>

So the mass of the Ostriker filament with 1.5 pc length is:

In [48]:
print(ml.decompose(u.cgs.bases).to(solarmassperparsec)*1.5)

36.2725031676986 solMass / pc


In [49]:
(1e-13 * u.g / (u.cm)**3/const.m_p).decompose(u.cgs.bases)

<Quantity 5.97863741e+10 1 / cm3>